Use the Advanced Optimization benchmark from AC hugging face (https://huggingface.co/spaces/AccelerationConsortium/crabnet-hyperparameter)
- Optimize (minimize) y1
    - If y1 is greater than 0.2, the result is considered "bad" no matter how good the other values are
- Transfer learning: 
    - higher fidelity means more expensive computation 
    - treat fidelity1 = 0.5 as "source' and fidelity1 = 1.0 as "target"
- Multi-task: 
    - since y1 and y2 are correlated 
    - treat y1 as task1, then y2 as task 2

In [1]:
from baybe import Campaign
from baybe.objectives import SingleTargetObjective
from baybe.parameters import NumericalContinuousParameter, CategoricalParameter
from baybe.searchspace import SearchSpace
from baybe.targets import NumericalTarget
from baybe.constraints import ContinuousLinearInequalityConstraint
import numpy as np
import pandas as pd
import torch
# load the Advanced Optimization from AC huggingface
from gradio_client import Client
client = Client("AccelerationConsortium/crabnet-hyperparameter")

Loaded as API: https://accelerationconsortium-crabnet-hyperparameter.hf.space ✔


In [2]:
# define the function 
# def adv_opt(x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19, x20, c1, c2, c3): 
#     result = client.predict(
#         x1, # float (numeric value between 0.0 and 1.0) in 'x1' Slider component
# 		x2,	# float (numeric value between 0.0 and 1.0)	in 'x2' Slider component
# 		x3,	# float (numeric value between 0.0 and 1.0) in 'x3' Slider component
# 		x4,	# float (numeric value between 0.0 and 1.0) in 'x4' Slider component
# 		x5,	# float (numeric value between 0.0 and 1.0) in 'x5' Slider component
# 		x6,	# float (numeric value between 0.0 and 1.0) in 'x6' Slider component
# 		x7,	# float (numeric value between 0.0 and 1.0) in 'x7' Slider component
# 		x8,	# float (numeric value between 0.0 and 1.0) in 'x8' Slider component
# 		x9,	# float (numeric value between 0.0 and 1.0) in 'x9' Slider component
# 		x10,	# float (numeric value between 0.0 and 1.0) in 'x10' Slider component
# 		x11,	# float (numeric value between 0.0 and 1.0) in 'x11' Slider component
# 		x12,	# float (numeric value between 0.0 and 1.0000000000000002) in 'x12' Slider component
# 		x13,	# float (numeric value between 0.0 and 1.0) in 'x13' Slider component
# 		x14,	# float (numeric value between 0.0 and 1.0) in 'x14' Slider component
# 		x15,	# float (numeric value between 0.0 and 1.0) in 'x15' Slider component
# 		x16,	# float (numeric value between 0.0 and 1.0) in 'x16' Slider component
# 		x17,	# float (numeric value between 0.0 and 1.0) in 'x17' Slider component
# 		x18,	# float (numeric value between 0.0 and 1.0) in 'x18' Slider component
# 		x19,	# float (numeric value between 0.0 and 0.9999999999999998) in 'x19' Slider component
# 		x20,	# float (numeric value between 0.0 and 0.9999999999999998) in 'x20' Slider component
# 		c1,	# Literal['c1_0', 'c1_1'] in 'c1' Radio component
# 		c2,	# Literal['c2_0', 'c2_1'] in 'c2' Radio component
# 		c3,	# Literal['c3_0', 'c3_1', 'c3_2'] in 'c3' Radio component
# 		0.5,	# float (numeric value between 0.0 and 1.0) in 'fidelity1' Slider component
# 		api_name="/predict",
#     )
#     return result['data'][0][0]			# return y1 value only


def adv_opt(**params):
    result = client.predict(
        params['x1'],  # float (numeric value between 0.0 and 1.0) in 'x1' Slider component
        params['x2'],  # float (numeric value between 0.0 and 1.0) in 'x2' Slider component
        params['x3'],  # float (numeric value between 0.0 and 1.0) in 'x3' Slider component
        params['x4'],  # float (numeric value between 0.0 and 1.0) in 'x4' Slider component
        params['x5'],  # float (numeric value between 0.0 and 1.0) in 'x5' Slider component
        params['x6'],  # float (numeric value between 0.0 and 1.0) in 'x6' Slider component
        params['x7'],  # float (numeric value between 0.0 and 1.0) in 'x7' Slider component
        params['x8'],  # float (numeric value between 0.0 and 1.0) in 'x8' Slider component
        params['x9'],  # float (numeric value between 0.0 and 1.0) in 'x9' Slider component
        params['x10'], # float (numeric value between 0.0 and 1.0) in 'x10' Slider component
        params['x11'], # float (numeric value between 0.0 and 1.0) in 'x11' Slider component
        params['x12'], # float (numeric value between 0.0 and 1.0000000000000002) in 'x12' Slider component
        params['x13'], # float (numeric value between 0.0 and 1.0) in 'x13' Slider component
        params['x14'], # float (numeric value between 0.0 and 1.0) in 'x14' Slider component
        params['x15'], # float (numeric value between 0.0 and 1.0) in 'x15' Slider component
        params['x16'], # float (numeric value between 0.0 and 1.0) in 'x16' Slider component
        params['x17'], # float (numeric value between 0.0 and 1.0) in 'x17' Slider component
        params['x18'], # float (numeric value between 0.0 and 1.0) in 'x18' Slider component
        params['x19'], # float (numeric value between 0.0 and 0.9999999999999998) in 'x19' Slider component
        params['x20'], # float (numeric value between 0.0 and 0.9999999999999998) in 'x20' Slider component
        params['c1'],  # Literal['c1_0', 'c1_1'] in 'c1' Radio component
        params['c2'],  # Literal['c2_0', 'c2_1'] in 'c2' Radio component
        params['c3'],  # Literal['c3_0', 'c3_1', 'c3_2'] in 'c3' Radio component
        0.5,           # float (numeric value between 0.0 and 1.0) in 'fidelity1' Slider component
        api_name="/predict",
    )
    return result['data'][0][0]  # return y1 value only


WRAPPED_FUNCTION = adv_opt

In [3]:
# define and create the search space
parameters = [
    NumericalContinuousParameter(name="x1", bounds=(0.0, 1.0)), 
    NumericalContinuousParameter(name="x2", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x3", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x4", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x5", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x6", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x7", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x8", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x9", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x10", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x11", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x12", bounds=(0.0, 1.0000000000000002)),
    NumericalContinuousParameter(name="x13", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x14", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x15", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x16", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x17", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x18", bounds=(0.0, 1.0)),
    NumericalContinuousParameter(name="x19", bounds=(0.0, 0.9999999999999998)),
    NumericalContinuousParameter(name="x20", bounds=(0.0, 0.9999999999999998)),
    CategoricalParameter(name='c1', values=['c1_0', 'c1_1'], encoding="OHE"),
    CategoricalParameter(name='c2', values=['c2_0', 'c2_1'], encoding="OHE"),
    CategoricalParameter(name='c3', values=['c3_0', 'c3_1', 'c3_2'], encoding="OHE"),
]

constraints = [
    ContinuousLinearInequalityConstraint(parameters=["x19", "x20"], coefficients=[-1.0, 1.0], rhs=0.0),
    ContinuousLinearInequalityConstraint(parameters=["x6", "x15"], coefficients=[-1.0, -1.0], rhs=-1.0), 
]

searchspace = SearchSpace.from_product(parameters=parameters, constraints=constraints)

# define objective
objective = SingleTargetObjective(target=NumericalTarget(name="Target", mode="MIN"))

# create campaign
campaign = Campaign(searchspace=searchspace, objective=objective)

In [4]:
for k in range(20): 
    recommendation = campaign.recommend(batch_size=1)

    # target value are looked up via the botorch wrapper
    target_values = []
    for index, row in recommendation.iterrows():
        print(row.to_dict())
        target_values.append(WRAPPED_FUNCTION(**row.to_dict()))

    recommendation["Target"] = target_values

    campaign.add_measurements(recommendation)       


{'c1': 'c1_0', 'c2': 'c2_1', 'c3': 'c3_0', 'x1': 0.6819717246401968, 'x2': 0.7162759562466889, 'x3': 0.013891682458476717, 'x4': 0.9549370910026038, 'x5': 0.9160224591457397, 'x6': 0.31774658135015693, 'x7': 0.8209560238739013, 'x8': 0.1429822953195338, 'x9': 0.503144136259913, 'x10': 0.571284187602213, 'x11': 0.5926502563163258, 'x12': 0.27736886720702014, 'x13': 0.24914202581152056, 'x14': 0.0359729045006697, 'x15': 0.3138924951937346, 'x16': 0.31490609990387664, 'x17': 0.5053288289414744, 'x18': 0.1287617509090324, 'x19': 0.26367133287550976, 'x20': 0.6371388784003893}


c:\Users\MF\anaconda3\envs\BayBE\lib\site-packages\botorch\models\transforms\outcome.py:289: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ..\aten\src\ATen\native\ReduceOps.cpp:1807.)
  stdvs = Y.std(dim=-2, keepdim=True)
c:\Users\MF\anaconda3\envs\BayBE\lib\site-packages\botorch\models\utils\assorted.py:194: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ..\aten\src\ATen\native\ReduceOps.cpp:1807.)
  Ymean, Ystd = torch.mean(Y, dim=-2), torch.std(Y, dim=-2)


{'c1': 'c1_1', 'c2': 'c2_0', 'c3': 'c3_1', 'x1': 1.0, 'x2': 0.0, 'x3': 0.999999999998665, 'x4': 0.0, 'x5': 5.859273978338507e-13, 'x6': 0.9999999999734767, 'x7': 0.0, 'x8': 0.999999999997967, 'x9': 9.357997484945513e-10, 'x10': 0.0, 'x11': 0.9999999999994005, 'x12': 1.0000000000000002, 'x13': 0.9999999994818111, 'x14': 0.9999999744846613, 'x15': 8.772926796563261e-13, 'x16': 0.9999999999980727, 'x17': 0.9999999998727145, 'x18': 0.9999999999975427, 'x19': 0.9999999999999998, 'x20': 0.9999999999999998}


AppError: The upstream Gradio app has raised an exception but has not enabled verbose error reporting. To enable, set show_error=True in launch().

### Calling to API and make calculation

In [5]:
result = client.predict(
		0.222,	# float (numeric value between 0.0 and 1.0) in 'x1' Slider component
		0.5,	# float (numeric value between 0.0 and 1.0)	in 'x2' Slider component
		0.445,	# float (numeric value between 0.0 and 1.0) in 'x3' Slider component
		0.1,	# float (numeric value between 0.0 and 1.0) in 'x4' Slider component
		0.5,	# float (numeric value between 0.0 and 1.0) in 'x5' Slider component
		0,	# float (numeric value between 0.0 and 1.0) in 'x6' Slider component
		0,	# float (numeric value between 0.0 and 1.0) in 'x7' Slider component
		0,	# float (numeric value between 0.0 and 1.0) in 'x8' Slider component
		0,	# float (numeric value between 0.0 and 1.0) in 'x9' Slider component
		0,	# float (numeric value between 0.0 and 1.0) in 'x10' Slider component
		0,	# float (numeric value between 0.0 and 1.0) in 'x11' Slider component
		0,	# float (numeric value between 0.0 and 1.0000000000000002) in 'x12' Slider component
		0,	# float (numeric value between 0.0 and 1.0) in 'x13' Slider component
		0,	# float (numeric value between 0.0 and 1.0) in 'x14' Slider component
		0,	# float (numeric value between 0.0 and 1.0) in 'x15' Slider component
		0,	# float (numeric value between 0.0 and 1.0) in 'x16' Slider component
		0,	# float (numeric value between 0.0 and 1.0) in 'x17' Slider component
		0,	# float (numeric value between 0.0 and 1.0) in 'x18' Slider component
		0,	# float (numeric value between 0.0 and 0.9999999999999998) in 'x19' Slider component
		0,	# float (numeric value between 0.0 and 0.9999999999999998) in 'x20' Slider component
		"c1_0",	# Literal['c1_0', 'c1_1'] in 'c1' Radio component
		"c2_0",	# Literal['c2_0', 'c2_1'] in 'c2' Radio component
		"c3_0",	# Literal['c3_0', 'c3_1', 'c3_2'] in 'c3' Radio component
		1.0,	# float (numeric value between 0.0 and 1.0) in 'fidelity1' Slider component
		api_name="/predict"
)
result

{'headers': ['y1', 'y2', 'y3', 'y4'],
 'data': [[0.6187509747504929,
   1.1785289373023509,
   331.66459280986527,
   9993118.024114504]],
 'metadata': None}

In [7]:
# get y1 only
result['data'][0][0]

0.6187509747504929

### Wrap the API into a function

In [8]:
paras = {'c1': 'c1_1', 'c2': 'c2_0', 'c3': 'c3_2', 'x1': 0.6118901958577212, 'x2': 0.4111829149245071, 'x3': 0.9669932071676511, 'x4': 0.4011947349568585, 'x5': 0.5327450665659677, 'x6': 0.21291356632372327, 'x7': 0.13115749846280078, 'x8': 0.3032873297687223, 'x9': 0.37433820950967467, 'x10': 0.6643725999349213, 'x11': 0.9719434718766504, 'x12': 0.20871047682655874, 'x13': 0.4884984758589027, 'x14': 0.5908026063989489, 'x15': 0.1366531028890536, 'x16': 0.7450997611208788, 'x17': 0.7656059548378924, 'x18': 0.8665776018638673, 'x19': 0.2905417268369886, 'x20': 0.8706679030815925}
adv_opt(**paras)
# WRAPPED_FUNCTION(**paras)

AppError: The upstream Gradio app has raised an exception but has not enabled verbose error reporting. To enable, set show_error=True in launch().